In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=False)
%cd 'drive/My Drive'
%cd 'Vol2SegGAN' ## project path

Mounted at /content/drive
/content/drive/My Drive
/content/drive/My Drive/IBSR/3DUnet_GAN_MIPS_Patch/15_Github_Vol2SegGAN


In [ ]:
!pip install tensorflow-gpu==2.1
!pip install tensorflow-addons==0.9.1
!pip install time
!pip install SimpleITK
!pip install natsort
!pip install nibabel as nib

In [ ]:
import tensorflow.compat.v1 as tf
from random import shuffle
from natsort import natsorted, ns
from data_load import DataGenerator
import glob
import os
import config
FLAGS = tf.app.flags.FLAGS

def data(data_selected):

    path = FLAGS.data_test+data_selected+"/"
        
    data_list = natsorted(os.listdir(path), alg=ns.PATH | ns.IGNORECASE)
    #shuffle(data_list)
    
    path=path+'*/'
    # images lists
    t1_list = sorted(glob.glob(path+'*t1_strip_registration.nii.gz'))
    seg_list = sorted(glob.glob(path+'*segm_registration_round_class.nii.gz'))#sorted(glob.glob(path+'*segm.nii.gz'))
    
    print(t1_list)
    veri=[]
    for i in data_list:
        i=int(i)-1
        veri.append([t1_list[i], seg_list[i]])
    return veri

sets = {'train': [], 'valid': [], 'test': []}
sets['test']=data(data_selected='test')

train_gen = DataGenerator(sets['test'],batch_size=FLAGS.batch_size,shuffle=False)
# valid_gen = DataGenerator(sets['valid'], augmentation=True, patch_size=128, n_patches=1)


['../test/1/t1_strip_registration.nii.gz', '../test/2/t1_strip_registration.nii.gz', '../test/3/t1_strip_registration.nii.gz', '../test/4/t1_strip_registration.nii.gz', '../test/5/t1_strip_registration.nii.gz', '../test/6/t1_strip_registration.nii.gz']
Data Loaded...


'\nimport matplotlib.pyplot as plt\na=0\nfor Xbatch, Ybatch in train_gen:\n    print(Xbatch.shape)\n    print(Ybatch.shape)\n    a=a+1\nplt.imshow(Xbatch[1,:,:,30,0])    \nprint(a)\n'

In [ ]:
import tensorflow.compat.v1 as tf
import os
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model,model_from_json
import SimpleITK as sitk
import json
import time
from model import Vol2SegGAN
from utils2 import  get_dsc

def tic():
    global _start_time
    _start_time = time.time()

def toc():
    t_sec = round(time.time() - _start_time)
    (t_min, t_sec) = divmod(t_sec,60)
    (t_hour,t_min) = divmod(t_min,60)
    time='Time: {}saat:{}dk:{}sn'.format(t_hour,t_min,t_sec)
    print(time)


def save_prediction(pred_array,name):#, patch_size, result_path):
    pred_array = np.squeeze(pred_array)
    pred_array = np.argmax(pred_array, axis=3).astype(np.float32)
    result_img = pred_array#sitk.GetImageFromArray(pred_array)
    path =FLAGS.data_test+str(name)+'/t1_strip_registration_Normalize.nii.gz'
    arr = nib.load(path)
    affine_ana = arr.affine
    new_image = nib.Nifti1Image(result_img, affine_ana)
    nib.save(new_image, FLAGS.resuts_path+str(name) + "_pred.nii.gz")

def test():
    path=FLAGS.save_model
    patch_size = list(map(int, FLAGS.patch_size.split(",")))
    px =patch_size[0]
    py =patch_size[1]
    pz =patch_size[2] 
      
    imShape = (px, py, pz, 1) 
    gtShape = (px, py, pz, 4)

    gan = Vol2SegGAN(imShape, gtShape, Nfilter_start=FLAGS.Nfilter_start,depth=FLAGS.depth)
    generator = gan.Generator()
    for i in range(1,4):
      epoch=i*160
      weights=path+str(epoch)+'_generator.h5' 
      generator.load_weights(weights)
      print('==epoch=='+str(epoch))      
      a=0
      test_gen = DataGenerator(sets['test'], batch_size=1,shuffle=False)
      for Xbatch, Ybatch in test_gen:  
        a+=1
        print(a)
        tic()    
        tmp_label =generator.predict(Xbatch)
        dsc = get_dsc(labels=Ybatch,predictions=tmp_label)
        if (epoch==480):
          save_prediction(tmp_label,a)
        test_status = "nBackground:\t\t{:0.4f}\n" + \
            "CSF:\t{:0.4f}\n" + \
            "GM:\t\t{:0.4f}\n" + \
            "WM:\t\t{:0.4f}\n" 
        print(test_status.format(dsc[0],
            dsc[1],
            dsc[2],
            dsc[3]
        )) 
        toc()

def main():    
    test()
    print("done!")

if __name__ == '__main__':
    main()